In [ ]:
from astropy.io import fits
from matplotlib import pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import ImageGrid
from IPython.display import clear_output
from IPython.html import widgets 
from IPython.display import display, clear_output
from plotly.widgets import GraphWidget
from IPython.html.widgets import FloatProgress
%matplotlib inline

In [ ]:
drange = np.linspace(-np.pi*4,np.pi*4,512)
data = np.sin(drange)

fit_coef = np.polynomial.chebyshev.chebfit(drange,data,30)
cheby = np.polynomial.chebyshev.chebval(drange,fit_coef)
plt.figure()
plt.plot(drange,data,"r")
plt.plot(drange,cheby,"b")
plt.show()

### Fit along columns per channel

In [ ]:
hdr = fits.open("/scratch/WORKSPACE/DDFacetTesting/data/beams/JVLA-L-centred-ll_ampl.fits")
img_data = hdr[0].data

reconstructed = np.zeros(img_data.shape)
POLY_ORDER = 70
cheby_fits = np.zeros([img_data.shape[0], img_data.shape[1], POLY_ORDER+1])
col_range = np.arange(img_data.shape[2])
pbar_fit = FloatProgress(min=0, max=100)
display(pbar_fit)
for ch in range(img_data.shape[0]):
    pbar_fit.value = ch / float(img_data.shape[0]) * 100.0
    for row in range(img_data.shape[1]):
        cheby_fits[ch,row,:] = np.polynomial.chebyshev.chebfit(col_range,
                                                               img_data[ch,row,:],POLY_ORDER)
        reconstructed[ch,row,:] = np.polynomial.chebyshev.chebval(col_range,
                                                                  cheby_fits[ch,row,:])
print "Done fitting..."

### Fit along channel

In [ ]:
POLY_ORDER = 36
img_corr = "RL"

hdr = fits.open("/scratch/JVLA_HOLOGRAPHY/JVLA-L/ant10"+img_corr+"real.fits")
img_data_real = hdr[0].data
hdr.close()
hdr = fits.open("/scratch/JVLA_HOLOGRAPHY/JVLA-L/ant10"+img_corr+"imag.fits")
img_data_imag = hdr[0].data
hdr.close()

flattened_real_view = img_data_real.reshape((img_data_real.shape[0],
                                  img_data_real.shape[1]*img_data_real.shape[2]))
flattened_imag_view = img_data_imag.reshape((img_data_imag.shape[0],
                                  img_data_imag.shape[1]*img_data_imag.shape[2]))
pxrange = np.arange(flattened_real_view.shape[1])
chrange = np.arange(flattened_real_view.shape[0])

flattened_real_reconstructed = np.zeros(flattened_real_view.shape)
flattened_imag_reconstructed = np.zeros(flattened_imag_view.shape)
cheby_real_coef = np.zeros([flattened_real_view.shape[1],POLY_ORDER+1])
cheby_imag_coef = np.zeros([flattened_imag_view.shape[1],POLY_ORDER+1])

pbar_fit = FloatProgress(min=0, max=100)
display(pbar_fit)
for px in pxrange:
    pbar_fit.value = px / float(flattened_real_view.shape[1]) * 100.0
    cheby_real_coef[px,:] = np.polynomial.chebyshev.chebfit(chrange,flattened_real_view[:,px],POLY_ORDER)
    flattened_real_reconstructed[:,px] = np.polynomial.chebyshev.chebval(chrange,cheby_real_coef[px,:])
    cheby_imag_coef[px,:] = np.polynomial.chebyshev.chebfit(chrange,flattened_imag_view[:,px],POLY_ORDER)
    flattened_imag_reconstructed[:,px] = np.polynomial.chebyshev.chebval(chrange,cheby_imag_coef[px,:])
reconstructed_real = flattened_real_reconstructed.reshape(img_data_real.shape)
reconstructed_imag = flattened_imag_reconstructed.reshape(img_data_imag.shape)

diff_img_real = img_data_real - reconstructed_real
diff_img_imag = img_data_imag - reconstructed_imag
print "Done fitting..."

In [ ]:
channel_slider = widgets.IntSlider()
channel_slider.min= 0
channel_slider.max= flattened_real_view.shape[0]-1
channel_slider.value = 0
channel_slider.description = 'Channel'
F = plt.figure(1,(15,15))
grid = ImageGrid(F, 111,  # similar to subplot(111)
        nrows_ncols=(2, 3),
        direction="row",
        axes_pad=0.5,
        add_all=True,
        label_mode="1",
        share_all=True,
        cbar_location="right",
        cbar_mode="each",
        cbar_size="3%")
im0 = grid[0].imshow(img_data_real[channel_slider.value], interpolation="nearest", cmap="cubehelix")
im1 = grid[1].imshow(reconstructed_real[channel_slider.value], interpolation="nearest", cmap="cubehelix")
im2 = grid[2].imshow(diff_img_real[channel_slider.value], interpolation="nearest", cmap="cubehelix")
im3 = grid[3].imshow(img_data_imag[channel_slider.value], interpolation="nearest", cmap="cubehelix")
im4 = grid[4].imshow(reconstructed_imag[channel_slider.value], interpolation="nearest", cmap="cubehelix")
im5 = grid[5].imshow(diff_img_imag[channel_slider.value], interpolation="nearest", cmap="cubehelix")
grid[0].set_title("Beam RE")
grid[1].set_title("Reconstructed RE")
grid[2].set_title("RE difference")
grid[3].set_title("Beam IM")
grid[4].set_title("Reconstructed IM")
grid[5].set_title("IM difference")

grid[0].cax.colorbar(im0)
grid[1].cax.colorbar(im1)
grid[2].cax.colorbar(im2)
grid[3].cax.colorbar(im3)
grid[4].cax.colorbar(im4)
grid[5].cax.colorbar(im5)
def animate():
    clear_output(wait=True)
    im0.set_data(img_data_real[channel_slider.value])
    im1.set_data(reconstructed_real[channel_slider.value])
    im2.set_data(diff_img_real[channel_slider.value])
    im3.set_data(img_data_imag[channel_slider.value])
    im4.set_data(reconstructed_imag[channel_slider.value])
    im5.set_data(diff_img_imag[channel_slider.value])
    grid[0].cax.colorbar(im0)
    grid[1].cax.colorbar(im1)
    grid[2].cax.colorbar(im2)
    grid[3].cax.colorbar(im3)
    grid[4].cax.colorbar(im4)
    grid[5].cax.colorbar(im5)
    plt.draw()
    display(F)
channel_slider.on_trait_change(animate, 'value')
display(channel_slider)